# Interview Paper v1.1

**Max Wu

*2017-02-24
- [Max]: Just update bs4 call to remove a warning after environment migration.
*2015-08-22
- [Max]: Change TOP_NUM to 2000 to take a trial.

In [1]:
import os, json
from pandas import DataFrame, Series
import pandas as pd; import numpy as np
import nltk, re, string
from bs4 import BeautifulSoup
TOP_NUM = 2000

In [3]:
#setup train-set 
path_g = 'training-data/group-ppl-0822-orig.json'
rec_g = [json.loads(line) for line in open (path_g, 'r',1)]
'Total number of g_json:' + str(rec_g[0]['total'])

tks_g = [[issue['fields']['project']['key'], issue['fields']['summary'], issue['fields']['description']] for issue in rec_g[0]['issues']]

frame_g = DataFrame(tks_g, columns=['prj', 'sum', 'desc'])

print frame_g.index
frame_g.head()

RangeIndex(start=0, stop=220, step=1)


,prj,sum,desc
0,TONTC,(CIG-401)mcast traffic can not pass ONT when c...,"Configure mcast service on ONT bridge port, VL..."
1,TONTC,(CIG-397)L3 mcast configuration is missed on C...,configure mcast service on ONT L3 port.\r\nOMC...
2,TONTC,(CIG-396)DS L2 unicast traffic can not pass ONT,Configure 7 data service and 1 mcast service o...
3,TONTC,(CIG-395) product code for R4.1.52.002 is wrong,We received R4.1.52.002 on 14th August. \r\nAf...
4,TONTC,(CIG-389) CIG does not report '1 BOOT' after r...,Calling Reboot RPC via TR69 to reboot CIG.\r\n...


In [6]:
#Utilities
def normalized_words(article_text):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stop_words = nltk.corpus.stopwords.words('english')
    words   = []
    oneline = article_text.replace('\n', ' ')
    soup = BeautifulSoup(oneline.strip(), "lxml")
    cleaned = soup.get_text()
    toks1   = cleaned.split()
    for t1 in toks1:
        translated = regex.sub('', t1)
        toks2 = translated.split()
        for t2 in toks2:
            t2s = t2.strip().lower()
            #if  t2s in stop_words:
            if t2s in stop_words or t2s.isdigit():
                pass
            else:
                words.append(t2s)
    return words

tokens = normalized_words(frame_g['desc'][2])
#tokens

#Return a list with all words shown in all items
def collect_all_words(items):
    all_words = []
    for item in items:
        for w in item:
            all_words.append(w)
    return list(set (all_words))

def identify_top_words(all_words):
    freq_dist = nltk.FreqDist(w.lower() for w in all_words)
    return freq_dist.keys()[:TOP_NUM]

#Count the top  words
desc_list = [normalized_words(desc) for desc in frame_g['desc']]
top_words = identify_top_words (collect_all_words(desc_list))

#$TOP_NUM words
len(top_words)

#read desc and generate hash against top_words with true/false judgement
def features(desc, top_words):
    word_s = set(normalized_words(desc))
    features ={}
    for wd in top_words:
        features["w_%s" %wd] = (wd in word_s) 
    return features

len(features(frame_g['desc'][1], top_words))

2000

In [7]:
#Prepare test-set
path_m = 'test-data/Binbin-0722-orig.json'
rec_m = [json.loads(line) for line in open (path_m, 'r',1)]
'Total number of g_json:' + str(rec_m[0]['total'])

tks_m = [[issue['fields']['project']['key'], issue['fields']['summary'], issue['fields']['description']] for issue in rec_m[0]['issues']]

frame_m = DataFrame(tks_m)

print frame_m.index
frame_m.head()

RangeIndex(start=0, stop=117, step=1)


,0,1,2
0,TONTT,The traffic will be lost even the pbit match t...,h5. Summary:\r\nThe traffic will be lost even ...
1,TONTT,MVR: The ME information in GIA include 3 MVR v...,h5. Summary:\r\nMVR: The ME information in GIA...
2,TONTT,ONT will transfer all the IGMP report packet t...,h5. Summary:\r\n ONT will transfer all th...
3,TONTC,(CIG-370)IPv6 echo request attack protect didn...,h5. Summary:\r\nONT version:R4.1.51.28\r\n\r\n...
4,TONTC,(CIG-369)The internet LED is always off.,h5. Summary:\r\nThe internet LED is always off...


In [8]:
#Train and test
def get_training_set(tickets):
    training_set = []
    for ticket in tickets:
        tup = (features(ticket[2], top_words), ticket[0])
        training_set.append(tup)
    return training_set

training_set = get_training_set(tks_g)
test_set = get_training_set(tks_m)

classifier = nltk.NaiveBayesClassifier.train(training_set)

print 'train on %d instances, test on %d instances' % (len(training_set), len(test_set)) 
classifier.show_most_informative_features(20)

#Test the classifier
print ("Accuracy = %f" %nltk.classify.accuracy(classifier, get_training_set(tks_m)))

catalogs = [classifier.classify(features(frame_m[2][idx], top_words))for idx in frame_m.index]
frame_m['guess'] = catalogs
frame_m['check'] = [frame_m[0][idx] == frame_m['guess'][idx] for idx in frame_m.index]
frame_m

train on 220 instances, test on 117 instances
Most Informative Features
                  w_summ = True              PXA : PREM   =     61.3 : 1.0
                w_ticket = True              PXA : PREM   =     61.3 : 1.0
                w_video1 = True              PXA : PREM   =     61.3 : 1.0
                 w_occur = True              PXA : PREM   =     61.3 : 1.0
                w_entity = True              PXA : PREM   =     61.3 : 1.0
                w_58to58 = True              PXA : PREM   =     61.3 : 1.0
               w_managed = True              PXA : PREM   =     61.3 : 1.0
         w_preconditions = False           TONTC : PREM   =     58.5 : 1.0
                 w_setup = False           TONTC : PREM   =     53.0 : 1.0
              w_returned = True            SXACC : PREM   =     46.0 : 1.0
         w_configuration = False           SXACC : PREM   =     46.0 : 1.0
            w_mvrprofile = True              PXA : PREM   =     36.8 : 1.0
               w_message = T

,0,1,2,guess,check
0,TONTT,The traffic will be lost even the pbit match t...,h5. Summary:\r\nThe traffic will be lost even ...,PREM,False
1,TONTT,MVR: The ME information in GIA include 3 MVR v...,h5. Summary:\r\nMVR: The ME information in GIA...,PREM,False
2,TONTT,ONT will transfer all the IGMP report packet t...,h5. Summary:\r\n ONT will transfer all th...,PREM,False
3,TONTC,(CIG-370)IPv6 echo request attack protect didn...,h5. Summary:\r\nONT version:R4.1.51.28\r\n\r\n...,TONTC,True
4,TONTC,(CIG-369)The internet LED is always off.,h5. Summary:\r\nThe internet LED is always off...,TONTC,True
5,TONTC,"(CIG-363)CIG missed the requirement R7.070, so...","CIG missed the requirement R7.070, some enhanc...",PREM,False
6,TONTC,(CIG-357)IPv6: ONT don's support the IPv6 echo...,h5. Summary:\r\nIPv6: ONT don's support the IP...,PREM,False
7,TONTC,(CIG-356)The default WAN address mode is SLAAC...,h5. Summary:\r\nThe default WAN address mode i...,PREM,False
8,TONTC,(CIG-350)ONT has the default value for the LAN...,h5. Summary:\r\nONT has the default value for ...,PREM,False
9,TONTC,(CIG-349)Login GUI failed after disable the du...,h5. Summary:\r\nLogin GUI failed after disable...,TONTC,True
